In [3]:
import paho.mqtt.client as mqtt
import json
import time
import pandas as pd
import plotly.express as px


In [ ]:
# Configuration
COLLECTION_TIME = 30  # Time in seconds to collect data 
OUTPUT_FILE = "collected_data.json"  

message_buffer = []

def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    client.subscribe("test/all_sensors")

def on_message(client, userdata, msg):
    try:
        data = json.loads(msg.payload)
        message_buffer.append(data)
    except json.JSONDecodeError:
        print("Failed to decode JSON message")

# Set up MQTT client
broker_url = "localhost" 
broker_port = 1883 

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.connect(broker_url, broker_port, 60)

client.loop_start()

start_time = time.time()
while time.time() - start_time < COLLECTION_TIME:
    print("Collecting data...")
    time.sleep(1)  

client.loop_stop()

with open(OUTPUT_FILE, "w") as f:
    json.dump(message_buffer, f, indent=4)



In [ ]:
# Load the JSON data from the file
with open('collected_data.json', 'r') as file:
    data = json.load(file)

# Extract relevant data
sensor_data = []
for entry in data:
    timestamp = entry.get('timestamp')
    sensors = entry.get('sensors', [])
    for sensor in sensors:
        sensor_name = sensor.get('sensor_name')
        sensor_type = sensor.get('sensor_type')
        values = sensor.get('values')
        sensor_data.append({
            'Timestamp': timestamp,
            'Sensor Name': sensor_name,
            'Sensor Type': sensor_type,
            'Values': values
        })

df = pd.DataFrame(sensor_data)

data = df[df["Sensor Type"] == 2]
data

,Timestamp,Sensor Name,Sensor Type,Values
2,1738230021408,Goldfish 3-axis Magnetic field sensor,2,"[0, 9.875, -47.75]"
20,1738230021510,Goldfish 3-axis Magnetic field sensor,2,"[0, 9.875, -47.75]"
38,1738230021610,Goldfish 3-axis Magnetic field sensor,2,"[0, 9.875, -47.75]"
56,1738230021711,Goldfish 3-axis Magnetic field sensor,2,"[0, 9.875, -47.75]"
74,1738230021814,Goldfish 3-axis Magnetic field sensor,2,"[0, 9.875, -47.75]"
...,...,...,...,...
1712,1738230031005,Goldfish 3-axis Magnetic field sensor,2,"[0, 9.875, -47.75]"
1730,1738230031105,Goldfish 3-axis Magnetic field sensor,2,"[0, 9.875, -47.75]"
1748,1738230031205,Goldfish 3-axis Magnetic field sensor,2,"[0, 9.875, -47.75]"
1766,1738230031305,Goldfish 3-axis Magnetic field sensor,2,"[0, 9.875, -47.75]"


In [ ]:
# Plotting function
def plot_sensor_values(df, sensor_name):
    """
    Plots the x, y, z values of the specified sensor.
    
    :param df: DataFrame containing the sensor data
    :param sensor_name: Name of the sensor to plot
    """
    # Filter the DataFrame for the specified sensor
    sensor_df = df[df['Sensor Type'] == sensor_name]
    
    # Extract the x, y, z values
    timestamps = sensor_df['Timestamp']
    values = sensor_df['Values'].tolist()
    x_values = [v[0] for v in values]
    y_values = [v[1] for v in values]
    z_values = [v[2] for v in values]
    
    # Create a DataFrame for plotting
    plot_df = pd.DataFrame({
        'Timestamp': timestamps,
        'X': x_values,
        'Y': y_values,
        'Z': z_values
    })
    
    # Plot the values using Plotly
    fig = px.line(plot_df, x='Timestamp', y=['X', 'Y', 'Z'], title=f'{sensor_name} Values Over Time')
    fig.show()

# Example usage
plot_sensor_values(df, 16)